In [1]:
#loading orders and users
import pandas as pd
orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")
print(orders)
print(users)


      order_id  user_id  restaurant_id  order_date  total_amount  \
0            1     2508            450  18-02-2023        842.97   
1            2     2693            309  18-01-2023        546.68   
2            3     2084            107  15-07-2023        163.93   
3            4      319            224  04-10-2023       1155.97   
4            5     1064            293  25-12-2023       1321.91   
...        ...      ...            ...         ...           ...   
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023        879.58   

                     restaurant_name  
0                  New Foods Chinese  
1     Ruchi Curry House Multicuisine  
2              Spice Kitchen Punjabi  
3             Darbar Kitche

In [2]:
#loading restaurants using mysql
import mysql.connector
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Root@123",
    database="Food_Delivery"
)

print(conn.is_connected())
restaurants = pd.read_sql(
    "SELECT * FROM restaurants",
    conn
)

print(restaurants)



True
     restaurant_id restaurant_name  cuisine  rating
0                1    Restaurant_1  Chinese     4.8
1                2    Restaurant_2   Indian     4.1
2                3    Restaurant_3  Mexican     4.3
3                4    Restaurant_4  Chinese     4.1
4                5    Restaurant_5  Chinese     4.8
..             ...             ...      ...     ...
495            496  Restaurant_496   Indian     3.1
496            497  Restaurant_497  Mexican     4.4
497            498  Restaurant_498  Chinese     3.9
498            499  Restaurant_499  Mexican     4.9
499            500  Restaurant_500  Chinese     3.2

[500 rows x 4 columns]


C:\Users\mypc\AppData\Local\Temp\ipykernel_29608\2638512606.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  restaurants = pd.read_sql(


In [38]:
#.changing order_date column data type string to standard format
##orders.dtypes


In [46]:
#.merge of orders,users,restaurants
final_df = (
    orders
    .merge(users, on='user_id', how='left')
    .merge(restaurants, on='restaurant_id', how='left')
)
final_df.shape
#created merged file final_dataset.csv 
final_df.to_csv("final_dataset.csv", index=False)

orders['order_date'] = pd.to_datetime(orders['order_date'], format='%d-%m-%Y')
orders.dtypes

order_id                    int64
user_id                     int64
restaurant_id               int64
order_date         datetime64[ns]
total_amount              float64
restaurant_name            object
dtype: object

In [42]:
#1.Which city has the highest total revenue (total_amount) from Gold members?  

final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum()


city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64

In [72]:
#2.Which cuisine has the highest average order value across all orders?

final_df.groupby('cuisine')['total_amount'] 
    .mean() \
    .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [73]:
#3.How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
high_spenders = (
    final_df.groupby('user_id')['total_amount']
    .sum()
    .reset_index()
)

count_users = high_spenders[high_spenders['total_amount'] > 1000].shape[0]
count_users


2544

In [74]:
#4.Which restaurant rating range generated the highest total revenue?
def rating_bucket(r):
    if 3.0 <= r <= 3.5:
        return '3.0–3.5'
    elif 3.6 <= r <= 4.0:
        return '3.6–4.0'
    elif 4.1 <= r <= 4.5:
        return '4.1–4.5'
    else:
        return '4.6–5.0'

final_df['rating_range'] = final_df['rating'].apply(rating_bucket)

final_df.groupby('rating_range')['total_amount'] 
    .sum() \
    .sort_values(ascending=False)

rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [19]:
#5.Among Gold members, which city has the highest average order value?

final_df[final_df['membership'] == 'Gold']  \
    .groupby('city')['total_amount'] \
     .mean() \
     .sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [76]:
#6.Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
final_df.groupby('cuisine').agg(
    restaurants=('restaurant_id', 'nunique'),
    revenue=('total_amount', 'sum')
).sort_values('restaurants')


,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [20]:
#7.What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
round(
    final_df[final_df['membership'] == 'Gold'].shape[0]
    * 100 / final_df.shape[0]
)


50

In [78]:
#8.Which restaurant has the highest average order value but less than 20 total orders?
restaurant_stats = final_df.groupby('restaurant_name_y').agg(
    orders=('order_id', 'count'),
    avg_value=('total_amount', 'mean')
)

restaurant_stats[restaurant_stats['orders'] < 20] \
    .sort_values('avg_value', ascending=False) \
    .head()



,orders,avg_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625


In [23]:
#9.Which combination contributes the highest revenue?
final_df.groupby(['membership', 'cuisine'])['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [88]:
restaurant_stats = final_df.groupby('restaurant_name_y').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
)

restaurant_stats_filtered = restaurant_stats[
    restaurant_stats['total_orders'] < 20
].sort_values('avg_order_value', ascending=False)

restaurant_stats_filtered.head()


,total_orders,avg_order_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625


In [89]:
final_df.groupby(['membership', 'cuisine'])['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [43]:

final_df['quarter'] = final_df['order_date'].dt.quarter

final_df.groupby('quarter')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [50]:
#How many total orders were placed by users with Gold membership?

final_df[final_df['membership'] == 'Gold']['order_id'].count()


np.int64(4987)

In [51]:
#What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

round(
    final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
)


1889367

In [52]:
#How many distinct users placed at least one order?

final_df['user_id'].nunique()


2883

In [53]:
#What is the average order value (rounded to 2 decimals) for Gold members?

round(
    final_df[final_df['membership'] == 'Gold']['total_amount'].mean(),
    2
)


np.float64(797.15)

In [54]:
#How many orders were placed for restaurants with rating ≥ 4.5?
final_df[final_df['rating'] >= 4.5]['order_id'].count()



np.int64(3374)

In [55]:
#How many orders were placed in the top revenue city among Gold members only?
final_df[
    (final_df['membership'] == 'Gold') &
    (final_df['city'] == 'Chennai')
]['order_id'].count()


np.int64(1337)